In [111]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

In [112]:
years = list(range(2025, 2021, -1))

In [113]:
years

[2025, 2024, 2023, 2022]

In [114]:
all_matches = []

In [115]:
standings_url = 'https://fbref.com/en/comps/9/Premier-League-Stats'

In [116]:
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]
    links = [link.get('href') for link in standings_table.find_all('a')]
    links = [link for link in links if '/squads/' in link]
    team_urls = [f'https://fbref.com{link}' for link in links]
    prev_season = soup.select('a.prev')[0].get('href')
    standings_url = f'https://fbref.com{prev_season}'
    for url in team_urls:
        team_name = url.split('/')[-1].replace('-Stats', '').replace('-', ' ')
        data = requests.get(url)
        matches = pd.read_html(data.text, match = "Scores & Fixtures ")[0]
        soup = BeautifulSoup(data.text)
        links = [link.get('href') for link in soup.find_all('a')]
        links = [link for link in links if link and 'all_comps/shooting/' in link]
        data = requests.get(f'https://fbref.com{links[0]}')
        shooting = pd.read_html(data.text, match = 'Shooting')[0]
        shooting.columns = shooting.columns.droplevel()

        try:
            team_data = matches.merge(shooting[['Date', 'Dist', 'Sh', 'SoT', 'FK', 'PK', 'PKatt']], on ="Date")
        except ValueError:
            continue
        team_data = team_data[team_data['Comp']=='Premier League']
        team_data['Season'] = year
        team_data['Team'] = team_name
        all_matches.append(team_data)
        time.sleep(20) # avoid being blocked by source website

/var/folders/q3/c8j9rrxj6vx_zl07g75p5xpc0000gn/T/ipykernel_72271/1969713109.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match = "Scores & Fixtures ")[0]
/var/folders/q3/c8j9rrxj6vx_zl07g75p5xpc0000gn/T/ipykernel_72271/1969713109.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match = 'Shooting')[0]
/var/folders/q3/c8j9rrxj6vx_zl07g75p5xpc0000gn/T/ipykernel_72271/1969713109.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match = "Scores & Fixtures ")[0]
/var/folders/q3/c8j9rrxj6vx_zl07g75p5xpc0000gn

In [104]:
url = 'https://fbref.com/en/comps/9/Premier-League-Stats'

response=requests.get(url)

print(response)

<Response [429]>


In [117]:
matches_df = pd.concat(all_matches)
matches_df.columns = [c.lower() for c in matches_df.columns]

In [118]:
matches_df.to_csv('matches.csv')

In [119]:
matches_df.shape

(2400, 28)

In [127]:
ips = matches_df[matches_df['team']=='Ipswich Town']

In [128]:
ips

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,dist,sh,sot,fk,pk,pkatt,season,team
0,2024-08-17,12:30,Premier League,Matchweek 1,Sat,Home,L,0,2,Liverpool,...,Match Report,NaN,16.0,7.0,2.0,0.0,0,0,2025,Ipswich Town
1,2024-08-24,15:00,Premier League,Matchweek 2,Sat,Away,L,1,4,Manchester City,...,Match Report,NaN,21.1,1.0,1.0,0.0,0,0,2025,Ipswich Town
3,2024-08-31,15:00,Premier League,Matchweek 3,Sat,Home,D,1,1,Fulham,...,Match Report,NaN,18.0,11.0,4.0,1.0,0,0,2025,Ipswich Town
4,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Away,D,0,0,Brighton,...,Match Report,NaN,16.6,6.0,1.0,0.0,0,0,2025,Ipswich Town
5,2024-09-21,15:00,Premier League,Matchweek 5,Sat,Away,D,1,1,Southampton,...,Match Report,NaN,14.1,13.0,6.0,0.0,0,0,2025,Ipswich Town
6,2024-09-29,14:00,Premier League,Matchweek 6,Sun,Home,D,2,2,Aston Villa,...,Match Report,NaN,18.4,15.0,5.0,0.0,0,0,2025,Ipswich Town
